In [2]:
import os
import subprocess
import pandas as pd
import platform
import pathlib
import numpy as np
from annotations import *
from extract_video_features import *
from extract_audio_features import *
import cv2
import seaborn as sns
from scipy.stats import kurtosis, skew
from sklearn.metrics import f1_score
import importlib
from tqdm import tqdm
import ordinal_classification as o_c
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from util_Valentina import runGridSearchClassifiers
from sklearn.model_selection import cross_val_predict

In [3]:
#get current directory
cwd = os.getcwd()

In [4]:
cwd

'/Users/valentinadiproietto/filrouge'

In [5]:
OpenFace_folder = '/Users/valentinadiproietto/OpenFace'
filename_annotations = 'https://docs.google.com/spreadsheets/d/1Rqu1sJiD-ogc4a6R491JTiaYacptOTqh6DKqhwTa8NA/gviz/tq?tqx=out:csv&sheet=Template'

In [6]:
Video_folder = '/Users/valentinadiproietto/Desktop/video_stress'

In [7]:
video_paths, video_names = get_videos(Video_folder)

In [7]:
video_names

['WIN_20210331_21_22_52_Pro',
 'WIN_20210329_14_13_45_Pro',
 'WIN_20210406_18_49_10_Pro',
 'WIN_20210408_14_11_32_Pro',
 'WIN_20210408_15_20_51_Pro',
 'WIN_20210404_10_58_27_Pro',
 'WIN_20210414_06_24_52_Pro',
 'WIN_20210406_15_06_15_Pro',
 'WIN_20210417_14_53_12_Pro',
 'WIN_20210413_15_38_01_Pro',
 'WIN_20210408_11_48_58_Pro',
 'WIN_20210408_16_04_32_Pro',
 'WIN_20210329_10_16_02_Pro',
 'WIN_20210323_19_17_40_Pro',
 'WIN_20210409_10_26_11_Pro',
 'Test_pour_AFPA',
 'WIN_20210405_15_09_16_Pro',
 'WIN_20210407_14_54_56_Pro_edit2',
 'WIN_20210406_21_05_52_Pro',
 'WIN_20210403_18_49_15_Pro',
 'WIN_20210408_14_02_19_Pro',
 'WIN_20210415_15_41_24_Pro',
 'WIN_20210406_18_35_52_Pro',
 'WIN_20210402_14_27_50_Pro',
 'WIN_20210407_09_04_05_Pro',
 'WIN_20210402_19_04_53_Pro',
 'WIN_20210416_08_06_54_Pro',
 'Video_1',
 'WIN_20210408_14_00_44_Pro',
 'WIN_20210404_21_41_12_Pro',
 'WIN_20210330_13_10_29_Pro']

In [8]:
list_dataframes = []
for i in video_names: 
    list_dataframes.append(create_dataframe_video('/Users/valentinadiproietto/OpenFace/processed/', i))


In [9]:
video_names.remove('WIN_20210329_14_13_45_Pro')
video_names.remove('WIN_20210402_14_27_50_Pro')


In [10]:
len(video_names)

29

In [11]:
list_df_max = []
for v_name in tqdm(video_names):
    df_annoted = get_df_video_with_annotations('/Users/valentinadiproietto/OpenFace/processed/', v_name, filename_annotations, "max")
    list_df_max.append(eliminate_features(df_annoted))


    

100%|██████████| 29/29 [02:12<00:00,  4.58s/it]


## RECUPERO LE PREDIZIONI SULLE TIME WINDOW

In [19]:
predict5s = pd.read_csv ('lstm_pred.csv')

In [20]:
predict5s = predict5s.drop('Unnamed: 0', axis = 1)
predict5s.columns = ['video_name', 'mean', 'std', 'min', '25%', '50%', '75%', 'max']

In [21]:
#recuperalo stress globale vero 
df_annotations_stress = pd.read_csv('annotations.csv')
df_annotations_stress = df_annotations_stress.drop(['1', '8', '9', '10', '11', '12', '17', '18'], axis =1)
df_annotations_stress

,video_name,stress_global
0,Test_pour_AFPA,1.0
1,Video_1,0.0
2,WIN_20210323_19_17_40_Pro,1.0
3,WIN_20210329_10_16_02_Pro,1.0
4,WIN_20210330_13_10_29_Pro,0.0
5,WIN_20210331_21_22_52_Pro,1.0
6,WIN_20210402_14_27_50_Pro,1.0
7,WIN_20210402_19_04_53_Pro,2.0
8,WIN_20210403_18_49_15_Pro,2.0
9,WIN_20210404_10_58_27_Pro,1.0


In [22]:
Xy = predict5s.merge(df_annotations_stress, how='inner', on=['video_name'])
X = Xy.iloc[:,:-1].set_index(['video_name'])
y = Xy.iloc[:,-1]
Xy

,video_name,mean,std,min,25%,50%,75%,max,stress_global
0,Test_pour_AFPA,0.696535,0.460029,0.0,0.0,1.0,1.0,1.0,1.0
1,Video_1,0.259202,0.438533,0.0,0.0,0.0,1.0,1.0,0.0
2,WIN_20210323_19_17_40_Pro,0.759891,0.427441,0.0,1.0,1.0,1.0,1.0,1.0
3,WIN_20210329_10_16_02_Pro,0.248447,0.432448,0.0,0.0,0.0,0.0,1.0,1.0
4,WIN_20210330_13_10_29_Pro,0.652842,0.476433,0.0,0.0,1.0,1.0,1.0,0.0
5,WIN_20210331_21_22_52_Pro,0.422222,0.494524,0.0,0.0,0.0,1.0,1.0,1.0
6,WIN_20210402_19_04_53_Pro,0.543011,0.498818,0.0,0.0,1.0,1.0,1.0,2.0
7,WIN_20210403_18_49_15_Pro,0.295082,0.456367,0.0,0.0,0.0,1.0,1.0,2.0
8,WIN_20210404_10_58_27_Pro,1.090596,0.865092,0.0,0.0,1.0,2.0,2.0,1.0
9,WIN_20210404_21_41_12_Pro,0.681641,0.466067,0.0,0.0,1.0,1.0,1.0,0.0


In [23]:
X

,mean,std,min,25%,50%,75%,max
video_name,,,,,,,
Test_pour_AFPA,0.696535,0.460029,0.0,0.0,1.0,1.0,1.0
Video_1,0.259202,0.438533,0.0,0.0,0.0,1.0,1.0
WIN_20210323_19_17_40_Pro,0.759891,0.427441,0.0,1.0,1.0,1.0,1.0
WIN_20210329_10_16_02_Pro,0.248447,0.432448,0.0,0.0,0.0,0.0,1.0
WIN_20210330_13_10_29_Pro,0.652842,0.476433,0.0,0.0,1.0,1.0,1.0
WIN_20210331_21_22_52_Pro,0.422222,0.494524,0.0,0.0,0.0,1.0,1.0
WIN_20210402_19_04_53_Pro,0.543011,0.498818,0.0,0.0,1.0,1.0,1.0
WIN_20210403_18_49_15_Pro,0.295082,0.456367,0.0,0.0,0.0,1.0,1.0
WIN_20210404_10_58_27_Pro,1.090596,0.865092,0.0,0.0,1.0,2.0,2.0


In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

pca = PCA()
pipe = Pipeline(steps=[('pca', pca), ('logistic', LogisticRegression(multi_class='multinomial', fit_intercept=True, random_state=42))])

models_list = [
                LogisticRegression(multi_class='multinomial', fit_intercept=True, random_state=42),
                Pipeline(steps=[('pca', pca), ('logistic', LogisticRegression(multi_class='multinomial', fit_intercept=True))]),
                KNeighborsClassifier(),
                Pipeline(steps=[('pca', pca), ('knn', KNeighborsClassifier())]),
                RandomForestClassifier(random_state = 42, n_jobs=-1)
                ]

parameters_list = [
                    {'C': [0.01, 0.05, 0.1, 0.5, 1, 2, 3, 4 , 5, 10], 'class_weight' : [None, 'balanced']},
                    {'pca__n_components': [1, 2, 3, 4],
                        'logistic__C': [0.01, 0.05, 0.1, 0.5, 1, 2, 3, 4 , 5, 10], 'logistic__class_weight' : [None, 'balanced']},
                    {'n_neighbors': [4, 5, 6, 7, 8, 9, 10, 11, 12,  15, 20], 'weights' : ['uniform', 'distance'], 'p': [1, 2]},
                    {'pca__n_components': [1, 2, 3, 4],
                        'knn__n_neighbors': [4, 5, 6, 7, 8, 9, 10, 11, 12,  15, 20], 'knn__weights' : ['uniform', 'distance'],                              'knn__p': [1, 2]},
                    {'n_estimators': [50, 100, 150, 200], 'max_depth':[3, 4, 5, 6, 10, 15, 20], 'class_weight':[None,'balanced']}
                    ]



In [25]:
best_result_tw5_glob, y_predict_tw5_glob, y_proba, result_list_tw5_glob = runGridSearchClassifiers(X, y, 5, models_list, parameters_list,output_predict=True, n_jobs=-1, verbose=True)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best estimator LogisticRegression(C=5, multi_class='multinomial', random_state=42)
Best results 0.40647619047619055
Best params {'C': 5, 'class_weight': None}
accuracy (mean, std) 0.4533333333333333 0.25785439474418287
f1 (mean, std) 0.40647619047619055 0.27332699880202915
balanced accuracy (mean, std) 0.4444444444444445 0.25337231668869736
precision (mean, std) 0.43499999999999994 0.31483681840315664
recall (mean, std) 0.4533333333333333 0.25785439474418287

Fitting 5 folds for each of 80 candidates, totalling 400 fits
Best estimator Pipeline(steps=[('pca', PCA(n_components=2)),
                ('logistic',
                 LogisticRegression(C=0.01, class_weight='balanced',
                                    multi_class='multinomial'))])
Best results 0.3448888888888889
Best params {'logistic__C': 0.01, 'logistic__class_weight': 'balanced', 'pca__n_components': 2}
accuracy (mean, std) 0.38 0.16275407487644938
f1 (mean, std

In [26]:
best_result_tw5_glob

{'best_estimator': Pipeline(steps=[('pca', PCA(n_components=2)),
                 ('knn', KNeighborsClassifier(n_neighbors=4, p=1))]),
 'best_score': 0.4353650793650794,
 'best_params': {'knn__n_neighbors': 4,
  'knn__p': 1,
  'knn__weights': 'uniform',
  'pca__n_components': 2},
 'mean_test_f1_score': 0.4353650793650794,
 'std_test_f1_score': 0.19534203969359829,
 'mean_test_accuracy_score': 0.4533333333333333,
 'std_test_accuracy_score': 0.18208667044996882,
 'mean_test_balanced_accuracy_score': 0.4333333333333333,
 'std_test_balanced_accuracy_score': 0.18392161508052052,
 'mean_test_precision': 0.5233333333333334,
 'std_test_precision': 0.25768197453450253,
 'mean_test_recall': 0.4533333333333333,
 'std_test_recall': 0.18208667044996882}

In [12]:
y_predict_tw5_glob
##ATTENZIONE NON PREDICE MAI I 2

array([1., 0., 1., 1., 1., 1., 0., 1., 0., 0., 1., 1., 1., 1., 1., 0., 0.,
       1., 0., 1., 0., 0., 1., 1., 1., 1., 0., 1., 0.])